# __Tweepy: Busca de tweets com Twitter API V1.1__

In [1]:
import tweepy, os
import pandas as pd
import tweepy as tw

In [6]:
keys = {
        'consumer_key': os.environ.get('API_KEY'),
        'consumer_secret': os.environ.get('API_SECRET'),
        'access_token_key': os.environ.get('ACCESS_TOKEN'),
        'access_token_secret': os.environ.get('ACCESS_TOKEN_SECRET'),
        'bearer_token': os.environ.get('BEARER_TOKEN')
    }

In [7]:
# autenticação
auth = tw.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token_key'], keys['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

## __Busca Padrão__

Permite a busca de tweets em um conjunto de tweets recentes, publicados dentro de um período de 7 dias.

In [36]:
# método de busca, retorna uma coleção de tweets relevantes para a query de pesquisa utilizada
tweets = api.search_tweets(
    q='python', # query contendo os termos que você quer pesquisar, pode usar hashtags e até operadores lógicos
    lang='pt', # filtragem por língua
    result_type='mixed', # parâmetro utilizado para escolher os tipos de tweets retornados, mixed mistura tweets mais recentes e tweets populares
    until='2022-09-01', # retorna tweets criados antes da data escolhida
    tweet_mode='extended', # faz com que os tweets não venham truncados
    count=100 # quantidade de tweets retornados por página
)

Resultado da busca

In [37]:
len(tweets) # foram encontrados no mínimo 100 tweets contendo o termo python

100

Conteúdo do JSON

In [38]:
tweets[0]._json

{'created_at': 'Wed Aug 31 23:57:16 +0000 2022',
 'id': 1565126606393311232,
 'id_str': '1565126606393311232',
 'full_text': 'RT @lara_mesquita: Brincando com o painel "siga o dinheiro" do Jota e @basedosdados , super legal.\nE a base de dados permite acessar os dad…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'lara_mesquita',
    'name': '🏴Lara Mesquita🗳',
    'id': 60428407,
    'id_str': '60428407',
    'indices': [3, 17]},
   {'screen_name': 'basedosdados',
    'name': 'Base dos Dados',
    'id': 1184334528837574656,
    'id_str': '1184334528837574656',
    'indices': [70, 83]}],
  'urls': []},
 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 


O limite máximo do valor do parâmetro é 100 tweets por página, entrentando, o tweepy nos permite trabalhar com paginação.

Isso quer dizer que podemos buscar por uma maior quantia de tweets, distribuídos em mais de uma página contendo no máximo 100 tweets cada.

In [9]:
tweets_cursor = tw.Cursor(
    api.search_tweets, # o método do tweepy que será utilizado
    q='python', # query contendo os termos que você quer pesquisar, pode usar hashtags e até operadores lógicos
    lang='pt', # filtragem por língua
    result_type='mixed', # parâmetro utilizado para escolher os tipos de tweets retornados, mixed mistura tweets mais recentes e tweets populares
    until='2022-09-01', # retorna tweets criados antes da data escolhida
    tweet_mode='extended', # faz com que os tweets não venham truncados
    count=100 # quantidade de tweets retornados por página
).items(200) # você pode limitar o número de tweets que quer coletar, caso nâo queira limitar é só não setar o parâmetro em items - .items()

In [10]:
type(tweets_cursor)

tweepy.cursor.ItemIterator

Esse métodos retorna um cursor do tweepy, basicamente um iterator, ao invés de um JSON contendo os tweets.

Para acessar os dados desse cursor é preciso iterar.

In [11]:
tweet_list = []

for tweet in tweets_cursor:
    tweet_list.append(tweet)
    
len(tweet_list)

200

In [15]:
tweet_list[0]._json

{'created_at': 'Wed Aug 31 23:57:16 +0000 2022',
 'id': 1565126606393311232,
 'id_str': '1565126606393311232',
 'full_text': 'RT @lara_mesquita: Brincando com o painel "siga o dinheiro" do Jota e @basedosdados , super legal.\nE a base de dados permite acessar os dad…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'lara_mesquita',
    'name': '🏴Lara Mesquita🗳',
    'id': 60428407,
    'id_str': '60428407',
    'indices': [3, 17]},
   {'screen_name': 'basedosdados',
    'name': 'Base dos Dados',
    'id': 1184334528837574656,
    'id_str': '1184334528837574656',
    'indices': [70, 83]}],
  'urls': []},
 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 

A partir desses JSON, você pode escolher as informações que quiser, criar e salvar um dataframe como csv ou parquet, ou até mesmo salvá-los como JSON.

Particulamente eu prefiro utilizar compreensão de listas durante a coleta, e não costumo utilizar limite de tweets.

Meus códigos costumam ficar assim:

In [16]:
tweets_json = [tweet._json for tweet in tw.Cursor(
    api.search_tweets,
    q='python AND "ciência de dados"',
    lang='pt',
    result_type='mixed',
    until='2022-09-01',
    tweet_mode='extended',
    count=100).items()]

print(f'Foram coletados {len(tweets_json)} tweets.')

Foram coletados 6 tweets.


In [17]:
tweets_json[0]

{'created_at': 'Wed Aug 31 22:47:42 +0000 2022',
 'id': 1565109098261188608,
 'id_str': '1565109098261188608',
 'full_text': 'Essa disciplina de métodos números em Python vai ser show, professor já levando pra ciência de dados. Velho, a ciência brasileira é capaz de usar IA pra identificar defeitos em placa mãe de computador/notebook, isso é muito foda.',
 'truncated': False,
 'display_text_range': [0, 229],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1190961862139154432,
  'id_str': '1190961862139154432',
  'name': 'Michell Santana',
  'screen_name': 'Deywindir',
  'location': 'Teixeira de Freitas, Brasil',
  '

## __Busca Premium__

Esse diferente tipo de busca utiliza a versão premium da API do Twitter e conta com dois _end points_ distintos:

  1. O _end point_ de busca de 30 dias que aumenta a janela de coleta vista na busca padrão;
  2. O _end point_ de busca completa que nos permite coletar tweets desde 2006.
  
Para utilizar esses tipos de buscas você precisa ter o _dev environment premium_ da API do Twitter para cada uma das duas. Eu tenho essa _feature_ por ter conseguido a versão de pesquisa acadêmica da API.

## Busca de 30 dias

In [8]:
label = os.environ.get('SANDBOX_LABEL')

In [13]:
tweets_30_day = api.search_30_day(
    label='research',
    query='python',
    fromDate='202209150000', # formato timestamp
    toDate='202209200000', # formato timestamp
    maxResults=100 # valor máximo página
)

len(tweets_30_day)

100

In [16]:
tweets_30_day = [tweet._json for tweet in tw.Cursor(
    api.search_30_day,
    label='research',
    query='python',
    fromDate='202209150000',
    toDate='202209200000',
    maxResults=100).items(200)]

len(tweets_30_day)

200

In [45]:
tweets_30_day[0]

{'created_at': 'Wed Aug 31 23:59:59 +0000 2022',
 'id': 1565127292841406465,
 'id_str': '1565127292841406465',
 'text': 'BrownieはPythonベースのフレームワークですか\nhttps://t.co/6RbEK7ld5f',
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1278550911829241858,
  'id_str': '1278550911829241858',
  'name': 'タカイ',
  'screen_name': '1minami2',
  'location': None,
  'url': None,
  'description': 'ネットワーク、ブロックチェーン、Dfinity勉強中',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 127,
  'friends_count': 258,
  'listed_count': 4,
  'favourites_count': 1858,
  'statuses_count': 750,
  'created_at': 'Thu Jul 02 04:48:58 +0000 2020',
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': False,
  'lang': None,
  'contributors_enabl

## Busca no Arquivo

Para esse tipo de busca os parâmetros são os mesmos, única diferença é que o limite do parâmetro "fromDate" vai até março de 2006 (primeiro ano do twitter).

In [21]:
tweets_arch = [tweet._json for tweet in tw.Cursor(
    api.search_full_archive,
    label='research',
    query='python',
    fromDate='201308150000',
    toDate='201309010000',
    maxResults=100).items(200)]

len(tweets_arch)

200

In [22]:
tweets_arch[0]

{'created_at': 'Sat Aug 31 23:59:41 +0000 2013',
 'id': 373958303325253632,
 'id_str': '373958303325253632',
 'text': '@RealZakWaddell   When I was dating, I enjoyed using Princess Bride, or Monte Python quotes as my test of worthyness!',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': 373956780071464960,
 'in_reply_to_status_id_str': '373956780071464960',
 'in_reply_to_user_id': 1424550680,
 'in_reply_to_user_id_str': '1424550680',
 'in_reply_to_screen_name': 'ZakW_EDU',
 'user': {'id': 259032188,
  'id_str': '259032188',
  'name': 'Sarah Reynolds',
  'screen_name': 'Sarah_Reynolds_',
  'location': None,
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 28,
  'friends_count': 68,
  'listed_count': 0,
  'favourites_count': 37,
  'statuses_count': 231,
  'created_at': 'Tue Mar 01 00:56:25 +0000 2011',
  'utc_offs